In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import confusion_matrix, f1_score

In [2]:
zip_file = keras.utils.get_file(
    fname="cora.tgz",
    origin="https://linqs-data.soe.ucsc.edu/public/lbc/cora.tgz",
    extract=True,
)
data_dir = os.path.join(os.path.dirname(zip_file), "cora")

168052/168052 [==============================] - 0s 0us/step


In [3]:
citations = pd.read_csv(
    os.path.join(data_dir, "cora.cites"),
    sep="\t",
    header=None,
    names=["target", "source"],
)
print("Citations shape:", citations.shape)

Citations shape: (5429, 2)


In [4]:
column_names = ["paper_id"] + [f"term_{idx}" for idx in range(1433)] + ["subject"]
papers = pd.read_csv(
    os.path.join(data_dir, "cora.content"), sep="\t", header=None, names=column_names,
)
print("Papers shape:", papers.shape)

Papers shape: (2708, 1435)


In [5]:
class_values = sorted(papers["subject"].unique())
class_idx = {name: id for id, name in enumerate(class_values)}
paper_idx = {name: idx for idx, name in enumerate(sorted(papers["paper_id"].unique()))}

papers["paper_id"] = papers["paper_id"].apply(lambda name: paper_idx[name])
citations["source"] = citations["source"].apply(lambda name: paper_idx[name])
citations["target"] = citations["target"].apply(lambda name: paper_idx[name])
papers["subject"] = papers["subject"].apply(lambda value: class_idx[value])

In [6]:
train_data, test_data = [], []

for _, group_data in papers.groupby("subject"):
    # Select around 80% of the dataset for training.
    random_selection = np.random.rand(len(group_data.index)) <= 0.8
    train_data.append(group_data[random_selection])
    test_data.append(group_data[~random_selection])

train_data = pd.concat(train_data).sample(frac=1)
test_data = pd.concat(test_data).sample(frac=1)

print("Train data shape:", train_data.shape)
print("Test data shape:", test_data.shape)

Train data shape: (2171, 1435)
Test data shape: (537, 1435)


In [7]:
feature_cols = [f"term_{idx}" for idx in range(1433)]
train_data_X = train_data[feature_cols]
train_data_y = train_data['subject']

test_data_X = test_data[feature_cols]
test_data_y = test_data['subject']

In [8]:
# Create Decision Tree classifer object
clf = DecisionTreeClassifier()
clf = clf.fit(train_data_X, train_data_y)
y_pred = clf.predict(test_data_X)

In [9]:
print("Accuracy:",metrics.accuracy_score(test_data_y, y_pred))
print(" ")
print("Confusion matrix: ")
print(confusion_matrix(test_data_y , y_pred))
print(" ")
print("F1 score: ", f1_score(test_data_y , y_pred, average = 'macro'))

Accuracy: 0.6573556797020484
 
Confusion matrix: 
[[ 45   4  10   0   3   5   2]
 [  2  56   3   1   4   3   4]
 [  4   7 114  11   2   5  13]
 [  6   1  15  57   0   2   6]
 [  2   1   9   3  26   0   3]
 [  2   1   2   1   0  15   5]
 [  7   2  17   7   7   2  40]]
 
F1 score:  0.6356272725197579
